In [2]:
from pathlib import Path

# Mapping

In [4]:
DATA_DIR = Path("../data")

RUN_DIRS = [run_dir 
 for run_dir in DATA_DIR.glob("*") 
 if run_dir.is_dir() 
 and len(list(run_dir.glob("*"))) != 0 ]

MAPPER_WUHAN_DIR = Path("../mapper_wuhan")
DEPLETION_DIR = Path("../human_depleted_fastqs/depletion_analysis")
GENOME_SARS_PATH = "../resources/genomes/MN908947_3.fa"

MAPPER_HUMAN_DIR = Path("../mapper_human")
GENOME_HUMAN_PATH = "../resources/genomes/hg38.fa"

MAPPER_HUMAN_SARS_DIR = Path("../mapper_human_sarscov2")
GENOME_HUMAN_SARS_PATH = "../resources/genomes/human_sars.fa"

## Mapping vs SARS-CoV Wuhan-Hu-1

In [4]:
print("MAPPING against Wuhan-Hu-1 the following runs:")
for run_dir in RUN_DIRS:
    
    # Prepare mapper outdir
    out_dir = MAPPER_WUHAN_DIR / run_dir.stem
    print(f"- {run_dir.stem} in {out_dir}")
    
    # Check if mapping already done:
    if out_dir.exists():
        print(f"The result folder {out_dir} exists already. Skipping...")
        continue
        
    # Prepare sequana command
    sequana_cmd = f"sequana_mapper --reference {GENOME_SARS_PATH} --input-directory {run_dir} --do-coverage --working-directory {out_dir} --mapper bowtie2"
    print(f"Sequana command: {sequana_cmd}")
    
    # Create sequana templates
    ! module load sequana/prod && {sequana_cmd}
    ! module load sequana/prod && cd {out_dir} && sbatch -q biomics -p biomics -A biomics mapper.sh

MAPPING against Wuhan-Hu-1 the following runs:
- illumina_resv_v2_ns in ../mapper/illumina_resv_v2_ns
Sequana command: sequana_mapper --reference /pasteur/projets/policy01/Biomics/resources/covid19/genomes/MN908947_3.fa --input-directory ../data/illumina_resv_v2_ns --do-coverage --working-directory ../mapper/illumina_resv_v2_ns --mapper bowtie2
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
INFO    [sequana.pipelines_common]:  Found 38 files matching your input  pattern (*fastq.gz)
INFO    [sequana.pipelines_common]:  Found 38 files matching your input  pattern (*fastq.gz)
Check the script in ../mapper/illumina_resv_v2_ns/mapper.sh as well as the configuration file in ../mapper/illumina_resv_v2_ns/config.yaml.

Once ready, execute the script mapper.sh using 

	cd ../mapper/illumina_resv_v2_ns; sbatch mapper.sh


INFO    [sequana.pipelines_co

## Mapping vs human

In [7]:
for run_dir in RUN_DIRS:
    
    # Prepare mapper outdir
    out_dir = MAPPER_HUMAN_DIR / run_dir.stem
    print(f"- {run_dir.stem} in {out_dir}")
    
    # Check if mapping already done:
    if out_dir.exists():
        print(f"The result folder {out_dir} exists already. Skipping...")
        continue
        
    # Prepare sequana command
    sequana_cmd = f"sequana_mapper --slurm-memory 30G --reference {GENOME_HUMAN_PATH} --input-directory {run_dir} --working-directory {out_dir} --mapper bowtie2"
    print(f"Sequana command: {sequana_cmd}")
    
    # Create sequana templates
    ! module load sequana/prod && {sequana_cmd}
    ! module load sequana/prod && cd {out_dir} && sbatch -q biomics -p biomics -A biomics mapper.sh

- illumina_resv_v2_ns in ../mapper_human_unmapped/illumina_resv_v2_ns
Sequana command: sequana_mapper --slurm-memory 30G --reference /pasteur/projets/policy01/Biomics/resources/genomes/hg38/hg38.fa --input-directory ../data/illumina_resv_v2_ns --working-directory ../mapper_human_unmapped/illumina_resv_v2_ns --mapper bowtie2
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
Traceback (most recent call last):
  File "/pasteur/projets/specific/Biomics/resources/condas/sequana_0_9_5/bin/sequana_mapper", line 8, in <module>
    sys.exit(main())
  File "/pasteur/projets/policy01/Biomics/resources/condas/sequana_0_9_5/lib/python3.7/site-packages/sequana_pipelines/mapper/main.py", line 82, in main
    manager.setup()
  File "/pasteur/projets/policy01/Biomics/resources/condas/sequana_0_9_5/lib/python3.7/site-packages/sequana-0.9.5-py3.7.egg/sequana/pipe

## In silico depletion

In [26]:
depletion_cmd = f"""
bwa mem {GENOME_HUMAN_PATH} ./art/data_1.fq ../art/data_2.fq -t 8 > temp.sam \
samtools view -bS temp.sam > temp.bam \
samtools view -b -f 12 -F 256 temp.bam > temp_bothReadsUnmapped.bam \
bioconvert bam2fastq temp_bothReadsUnmapped.bam test.fastq --force \
"""

for run_dir in RUN_DIRS:
    
    # Prepare mapper outdir
    DEPLETION_DIR.mkdir(exist_ok=True)
    out_dir = DEPLETION_DIR / run_dir.stem
    print(f"- {run_dir.stem} in {out_dir}")
    FASTQ_DIR = out_dir / "depleted_fastqs"
    FASTQ_DIR.mkdir(exist_ok=True)
    
    sequana_cmd = f"sequana_mapper --slurm-memory 30G --reference {GENOME_HUMAN_PATH} --input-directory {run_dir} --working-directory {out_dir} --mapper bwa"
    ! module load sequana/prod && {sequana_cmd}
    ! module load sequana/prod && cd {out_dir} && sbatch -q biomicspole -p biomicspole mapper.sh
    
    for bam in out_dir.glob("*/bwa_mem_mapping/*.bam"):
        print(bam)
        fastq = FASTQ_DIR / bam.with_suffix('.depleted.fastq').name
        samtools_cmd = f"samtools view -b -f 12 -F 256 {bam} > {bam.with_suffix('.depleted.bam')}"
        ! {samtools_cmd}
        bioconvert_cmd = f"module load bioconvert && bioconvert bam2fastq {bam.with_suffix('.depleted.bam')} {fastq} --force"
        ! {bioconvert_cmd}

- arbo_single_is in ../human_depleted_fastqs/arbo_single_is
../human_depleted_fastqs/arbo_single_is/4660_S3_L001/bwa_mem_mapping/4660_S3_L001.sorted.bam
HHH
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 59788 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 59788 reads
../human_depleted_fastqs/arbo_single_is/4798_S16_L001/bwa_mem_mapping/4798_S16_L001.sorted.bam
HHH
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 262 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 262 reads
../human_depleted_fastqs/arbo_single_is/4797_S17_L001/bwa_mem_mapping/4797_S17_L001.sorted.bam
HHH
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 314 reads
[M::bam2fq_mainloop] discarded 0 singletons
[M::bam2fq_mainloop] processed 314 reads
../human_depleted_fastqs/arbo_single_is/4656_S18_L001/bwa_mem_mapping/4656_S18_L001.sorted.bam
HHH
[M::bam2fq_mainloop] discarded 0

## Mapping Human and Sars-Cov

In [4]:
! cat {GENOME_SARS_PATH} {GENOME_HUMAN_PATH} > {GENOME_HUMAN_SARS_PATH}

In [6]:
for run_dir in RUN_DIRS:
    
    # Prepare mapper outdir
    run_dir = Path(run_dir)
    out_dir = MAPPER_HUMAN_SARS_DIR / run_dir.stem
    print(f"- {run_dir.stem} in {out_dir}")
    
    # Check if mapping already done:
    if out_dir.exists():
        print(f"The result folder {out_dir} exists already. Skipping...")
        continue
        
    # Prepare sequana command
    sequana_cmd = f"sequana_mapper --slurm-memory 30G --reference {GENOME_HUMAN_SARS_PATH} --input-directory {run_dir} --working-directory {out_dir} --mapper bowtie2"
    print(f"Sequana command: {sequana_cmd}")
    
    # Create sequana templates
    ! module load sequana/prod && {sequana_cmd}
    ! module load sequana/prod && cd {out_dir} && sbatch -q hubbioit -p hubbioit mapper.sh

- illumina_resv_v2_ns in ../mapper_human_sarscov2/illumina_resv_v2_ns
Sequana command: sequana_mapper --slurm-memory 30G --reference ../resources/human_sars.fa --input-directory ../data/illumina_resv_v2_ns --working-directory ../mapper_human_sarscov2/illumina_resv_v2_ns --mapper bowtie2
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
INFO    [sequana.pipelines_common]:  Welcome to Sequana pipelines suite (sequana.readthedocs.io)
INFO    [sequana.pipelines_common]:  Found 38 files matching your input  pattern (*fastq.gz)
INFO    [sequana.pipelines_common]:  Found 38 files matching your input  pattern (*fastq.gz)
Check the script in ../mapper_human_sarscov2/illumina_resv_v2_ns/mapper.sh as well as the configuration file in ../mapper_human_sarscov2/illumina_resv_v2_ns/config.yaml.

Once ready, execute the script mapper.sh using 

	cd ../mapper_human_sarscov2/illumina_resv_v2_ns; sbatch mapper.sh


INFO    [sequana.pipelines_common]:  A comp